In [1]:
#Pretty Display of Variables

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import libraries

In [47]:
#from plot_prop import plot_direction, plot_pointcloud, plot_segments, plot_projection, plot_all_projections
from prop_info import extreme_points, vect_blade, d_blade
from get_segments import blade_alone, get_segments_points, get_planes
from major_axis import get_major_axis

from parameters import get_hub_points, find_hub_radius
from plot_param import plot_hub
#from projections import projections_by_side, couple_all_planes, project_all_couples, points_to_project
#from plot_prop import plot_final_projections

import pandas as pd
import numpy as np

## Pre-processing

In [5]:
propeller_coords = pd.read_csv('aerostar_data.csv')

max_point, min_point, middle_point, highest_point, lowest_point = extreme_points(propeller_coords)

vect_length = vect_blade(max_point, min_point) 

dmiddle, dhighest, dlowest = d_blade(vect_length, middle_point, highest_point, lowest_point)

upper_blade, lower_blade = blade_alone(propeller_coords, vect_length, dmiddle)

vect_out, vect_side, hub_inner_radius = get_major_axis(propeller_coords, middle_point, vect_length)

## Projections

## Find parameters

    - Tip radius 
        Projection dans le sens de la longueur et ... ??
    - Blade twist 
        Project on planes as in projection
        Compute difference of angle between main directions of projections (PCA possible)

### Hub radius

In [57]:
hub_points = get_hub_points(propeller_coords, dmiddle, vect_length)
point_outer_radius, point_inner_radius = find_hub_radius(middle_point, hub_inner_radius, vect_side, hub_points)
hub_radius = point_outer_radius - middle_point  #du centre à l'exterieur
hub_radius_norm = np.linalg.norm(hub_radius)

In [59]:
hub_radius_norm

7.1330189878270165

### Chord length

In [60]:
chord_length = np.linalg.norm(highest_point - middle_point) - hub_radius_norm
chord_length

56.333310137030246

### Blade twist